# Loadings libraries and dataset

In [1]:
import pandas as pd
import numpy as np
import os
from functools import reduce

from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression, PLSCanonical
from sklearn.linear_model import LogisticRegression, Ridge
import random
from joblib import Parallel, delayed


import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from scipy.stats import pearsonr
from tqdm import tqdm
from nilearn.signal import clean

In [2]:
# Load participant-level data
df = pd.read_csv('../prevent_AD_data_dummy_jan_2023_fam_hist.csv', index_col=0)

# Load all time points (longitudinal visits)
timepoints_df = pd.read_csv('../all_time_points_01.27.23.csv')

# Extract PSCID list for participants with non-missing 50-month scores
pscid_df = timepoints_df.dropna(subset=['50'])[['CandID', 'PSCID_RBANS']]
pscid_list = pscid_df['PSCID_RBANS'].unique()
print(f"Number of unique PSCID_RBANS: {len(pscid_list)}")

# Merge relevant participant info with PSCID
info_cols = ['only_mother', 'CandID', 'Sex_Female', 'APOE']
infos = (
    df[info_cols]
    .drop_duplicates()
    .merge(pscid_df, on='CandID', how='left')
)

Number of unique PSCID_RBANS: 370


In [3]:
# Load canonical variates from CCA at each time point
cca_files = {
    "BL00": '../BL00_CCA_modes.csv',
    "FU12": '../FU12_CCA_modes.csv',
    "FU24": '../FU24_CCA_modes.csv',
    "FU36": '../FU36_CCA_modes.csv',
    "FU48": '../FU48_CCA_modes.csv'
}

cca_dfs = [pd.read_csv(fname, index_col=1) for fname in cca_files.values()]
all_cca = pd.concat(cca_dfs, keys=cca_files.keys()).drop(columns='Unnamed: 0')

# Maternal vs Paternal 

## Importing datasets

### Hippocampus

In [4]:
# Load hippocampal (HC) data from various time points
hc_files = {
    "BL00": '../HC_segmentation/csv/BL00_HC_left_right.csv',
    "FU12": '../HC_segmentation/csv/FU12_HC_left_right.csv',
    "FU24": '../HC_segmentation/csv/FU24_HC_left_right.csv',
    "FU36": '../HC_segmentation/csv/FU36_HC_left_right.csv',
    "FU48": '../HC_segmentation/csv/FU48_HC_left_right.csv'
}

# Read and store HC data for each time point
hc_dfs = [pd.read_csv(file, index_col=0) for file in hc_files.values()]

# Combine all HC data into a single DataFrame with multi-level index (timepoint + participant)
hc_volumes = pd.concat(hc_dfs, keys=hc_files.keys())

# Ensure that the HC data aligns with the canonical variates index (all_cca)
hc_volumes = hc_volumes.loc[all_cca.index]

# Print the shape to verify data alignment
print(hc_volumes.shape)


(912, 46)


In [5]:
# Reset the index and rename columns for clarity
hc_volumes = hc_volumes.reset_index().rename(columns={
    'level_0': 'visit',  # Rename the first index level to 'visit'
    'level_1': 'PSCID'         # Rename the second index level to 'PSCID'
})

# Merge the HC data with the 'infos' DataFrame based on 'PSCID'
hc_volumes = hc_volumes.merge(infos, left_on='PSCID', right_on='PSCID_RBANS', how='left')


In [6]:
# Extract base list of hippocampal column names from BL00_HC, excluding the last 4
col_HC = list(hc_volumes.iloc[:,2:-6].columns)

# Define columns to remove explicitly for clarity and safety
exclude_cols = [
    'Whole_hippocampal_body_left',
    'Whole_hippocampal_head_left',
    'Whole_hippocampus_left',
    'PSCID_left',
    'Whole_hippocampal_body_right',
    'Whole_hippocampal_head_right',
    'Whole_hippocampus_right',
    'PSCID_right',
]

# Remove columns only if they exist in the list
col_HC = [col for col in col_HC if col not in exclude_cols]

print(len(col_HC))


38


In [7]:
# Standardization and normalization
for col in col_HC:
    hc_volumes[col] = StandardScaler().fit_transform(hc_volumes[[col]])

### Default Network

In [8]:
# Load default network (DN) volume data from multiple time points
dn_files = {
    "BL00": '../BL00_DN.csv',
    "FU12": '../FU12_DN.csv',
    "FU24": '../FU24_DN.csv',
    "FU36": '../FU36_DN.csv',
    "FU48": '../FU48_DN.csv'
}

# Read and store DN data for each time point
dn_dfs = [pd.read_csv(file, index_col=0) for file in dn_files.values()]

# Combine all DN data into a single DataFrame with multi-level index (timepoint + participant)
dn_volumes = pd.concat(dn_dfs, keys=dn_files.keys())

# Ensure that the DN data aligns with the canonical variates index (all_cca)
dn_volumes = dn_volumes.loc[all_cca.index]

# Print the shape to verify data alignment
print(dn_volumes.shape)


(912, 91)


In [9]:
# Clean column names
dn_volumes.columns = [col.split('Networks_')[-1] if i > 0 else col for i, col in enumerate(dn_volumes.columns)]

# Store cleaned column names for reference
dn_cols = dn_volumes.columns

# Reset multi-index and rename to meaningful labels
dn_volumes = dn_volumes.reset_index().rename(columns={'level_0': 'visit', 'level_1': 'PSCID'})

# Merge with participant info using PSCID
dn_volumes = dn_volumes.merge(infos, left_on='PSCID', right_on='PSCID_RBANS', how='left')


In [10]:
# Standardization and normalization
for col in dn_cols:
    dn_volumes[col] = StandardScaler().fit_transform(dn_volumes[[col]])

In [11]:
def compute_pval(coefs, real_coefs):
    """
    Compute empirical one-sided p-values based on null distribution in `coefs`
    compared to observed values in `real_coefs`.

    Parameters:
    - coefs (pd.DataFrame): Null distribution for each coefficient (shape: permutations × variables).
    - real_coefs (list or np.ndarray): Observed coefficients (length must match number of columns in `coefs`).

    Returns:
    - dict: Variable name → p-value.
    """
    pvals = {}
    for col, obs_val in zip(coefs.columns, real_coefs):
        null_dist = coefs[col]

        if obs_val < 0:
            pval = (null_dist < obs_val).sum() / len(null_dist)
        else:
            pval = (null_dist > obs_val).sum() / len(null_dist)

        pvals[col] = pval

    return pvals


## Permutation

In [12]:
date = '25.05.13'

### Females

In [13]:
# Function to fit null models in parallel
def fit_null_model(X_scaled, y, seed):
    shuffled_y = np.random.default_rng(seed).permutation(y)
    model = LogisticRegression(solver='liblinear')
    model.fit(X_scaled, shuffled_y)
    return model.coef_[0]

# Storage for results
all_coefs_f = pd.DataFrame(columns=col_HC)
pval_dfs = []

# Begin permutation loop
for seed in tqdm(range(1000), desc="Permutations for female model"):
    random.seed(seed)

    # Stratified sampling
    group_indices = {
        'females_mat': np.where((hc_volumes.only_mother == 1) & (hc_volumes.Sex_Female == 1))[0],
        'females_pat': np.where((hc_volumes.only_mother == 0) & (hc_volumes.Sex_Female == 1))[0],
    }

    sampled_indices = np.concatenate([
        random.sample(list(group_indices['females_mat']), 50),
        random.sample(list(group_indices['females_pat']), 50),
    ])

    subset = hc_volumes.iloc[sampled_indices].sort_index()
    X_females = np.array(subset[col_HC])
    y_females = subset.only_mother.values

    # Fit real model
    model = LogisticRegression(solver='liblinear')
    model.fit(X_females, y_females)
    real_coefs = model.coef_[0]
    real_coefs_df = pd.DataFrame(model.coef_, columns=col_HC)
    all_coefs_f = pd.concat([all_coefs_f, real_coefs_df], ignore_index=True)
    
    # Build null distribution in parallel
    null_seeds = range(0,1000)
    null_coefs = Parallel(n_jobs=6)(delayed(fit_null_model)(X_females, y_females, s) for s in null_seeds)
    null_coefs_df = pd.DataFrame(null_coefs, columns=col_HC)

    # Compute p-values
    dict_pval = compute_pval(null_coefs_df, real_coefs)
    pval_df = pd.DataFrame({
        'coefs': list(dict_pval.keys()),
        f'pval_{seed}': list(dict_pval.values())
    })
    pval_dfs.append(pval_df)

# Save all coefficients
all_coefs_f.to_csv(f'{date}/brain_imaging/diagnostic_test/all_coefs_females_prevent_AD_hc_2025.csv', index=False)

# Combine p-values after loop
all_pvals_f = reduce(lambda left, right: left.merge(right, on='coefs'), pval_dfs)
# Save results
all_pvals_f.to_csv(f'{date}/brain_imaging/diagnostic_test/prevent_ad_hc_all_pvals_f_1000_2025.csv', index=False)


Permutations for female model: 100%|██████████| 1000/1000 [06:26<00:00,  2.59it/s]


### Males

In [14]:
# Function to fit null model with shuffled outcome
def fit_null_model(X_scaled, y, seed):
    shuffled_y = np.random.default_rng(seed).permutation(y)
    model = LogisticRegression(solver='liblinear')
    model.fit(X_scaled, shuffled_y)
    return model.coef_[0]

# Store results across permutations
all_coefs_m = pd.DataFrame(columns=col_HC)
pval_dfs_m = []

for seed in tqdm(range(1000), desc="Permutations for male model"):
    random.seed(seed)

    # Stratified sampling
    group_indices = {
        'males_mat':   np.where((hc_volumes.only_mother == 1) & (hc_volumes.Sex_Female == 0))[0],
        'males_pat':   np.where((hc_volumes.only_mother == 0) & (hc_volumes.Sex_Female == 0))[0],
    }

    sampled_indices = np.concatenate([
        random.sample(list(group_indices['males_mat']), 50),
        random.sample(list(group_indices['males_pat']), 50),
    ])

    subset = hc_volumes.iloc[sampled_indices].sort_index()
    X_males = np.array(subset[col_HC])
    y_males = subset.only_mother.values

    # Fit real model
    model = LogisticRegression(solver='liblinear')
    model.fit(X_males, y_males)
    real_coefs = model.coef_[0]
    real_coefs_df = pd.DataFrame(model.coef_, columns=col_HC)
    all_coefs_m = pd.concat([all_coefs_m, real_coefs_df], ignore_index=True)

    # Build null distribution in parallel
    null_seeds = range(0,1000)
    null_coefs = Parallel(n_jobs=6)(
        delayed(fit_null_model)(X_males, y_males, s) for s in null_seeds
    )
    null_coefs_df = pd.DataFrame(null_coefs, columns=col_HC)

    # Compute p-values
    dict_pval = compute_pval(null_coefs_df, real_coefs)
    pval_df = pd.DataFrame({
        'coefs': list(dict_pval.keys()),
        f'pval_{seed}': list(dict_pval.values())
    })
    pval_dfs_m.append(pval_df)

# Save all coefficients
all_coefs_m.to_csv(f'{date}/brain_imaging/diagnostic_test/all_coefs_males_prevent_AD_hc_2025.csv', index=False)

# Combine p-values after loop
all_pvals_m = reduce(lambda left, right: left.merge(right, on='coefs'), pval_dfs_m)
# Save results
all_pvals_m.to_csv(f'{date}/brain_imaging/diagnostic_test/prevent_ad_hc_all_pvals_m_1000_2025.csv', index=False)


Permutations for male model: 100%|██████████| 1000/1000 [08:04<00:00,  2.06it/s]


## Default Network

### Males

In [15]:
# Function to fit null model with shuffled outcome
def fit_null_model(X_scaled, y, seed):
    shuffled_y = np.random.default_rng(seed).permutation(y)
    model = LogisticRegression(solver='liblinear')
    model.fit(X_scaled, shuffled_y)
    return model.coef_[0]

# Store results across permutations
all_coefs_m = pd.DataFrame(columns=dn_cols)
pval_dfs_m = []

for seed in tqdm(range(1000), desc="Permutations for DN male model"):
    random.seed(seed)

    # Stratified sampling
    group_indices = {
        'males_mat':   np.where((dn_volumes.only_mother == 1) & (dn_volumes.Sex_Female == 0))[0],
        'males_pat':   np.where((dn_volumes.only_mother == 0) & (dn_volumes.Sex_Female == 0))[0],
    }

    sampled_indices = np.concatenate([
        random.sample(list(group_indices['males_mat']), 50),
        random.sample(list(group_indices['males_pat']), 50),
    ])

    subset = dn_volumes.iloc[sampled_indices].sort_index()
    X_males = np.array(subset[dn_cols])
    y_males = subset.only_mother.values

    # Fit real model
    model = LogisticRegression(solver='liblinear')
    model.fit(X_males, y_males)
    real_coefs = model.coef_[0]
    real_coefs_df = pd.DataFrame(model.coef_, columns=dn_cols)
    all_coefs_m = pd.concat([all_coefs_m, real_coefs_df], ignore_index=True)

    # Build null distribution in parallel
    null_seeds = range(0,1000)
    null_coefs = Parallel(n_jobs=6)(
        delayed(fit_null_model)(X_males, y_males, s) for s in null_seeds
    )
    null_coefs_df = pd.DataFrame(null_coefs, columns=dn_cols)

    # Compute p-values
    dict_pval = compute_pval(null_coefs_df, real_coefs)
    pval_df = pd.DataFrame({
        'coefs': list(dict_pval.keys()),
        f'pval_{seed}': list(dict_pval.values())
    })
    pval_dfs_m.append(pval_df)

# Save results
all_coefs_m.to_csv(f'{date}/brain_imaging/diagnostic_test/all_coefs_males_prevent_AD_dn_2025.csv', index=False)

# Combine p-values after loop
all_pvals_m = reduce(lambda left, right: left.merge(right, on='coefs'), pval_dfs_m)

# Save results
all_pvals_m.to_csv(f'{date}/brain_imaging/diagnostic_test/prevent_ad_dn_all_pvals_m_1000_2025.csv', index=False)


Permutations for DN male model:  36%|███▌      | 362/1000 [08:47<23:00,  2.16s/it]   /Users/chloesavignac/.conda/envs/masters_projects/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Permutations for DN male model: 100%|██████████| 1000/1000 [28:13<00:00,  1.69s/it]   


### Females

In [16]:
# Function to fit null model with shuffled outcome
def fit_null_model(X_scaled, y, seed):
    shuffled_y = np.random.default_rng(seed).permutation(y)
    model = LogisticRegression(solver='liblinear')
    model.fit(X_scaled, shuffled_y)
    return model.coef_[0]

# Store results across permutations
all_coefs_f = pd.DataFrame(columns=dn_cols)
pval_dfs_f = []

for seed in tqdm(range(1000), desc="Permutations for DN female model"):
    random.seed(seed)

    # Stratified sampling
    group_indices = {
        'females_mat': np.where((dn_volumes.only_mother == 1) & (dn_volumes.Sex_Female == 1))[0],
        'females_pat': np.where((dn_volumes.only_mother == 0) & (dn_volumes.Sex_Female == 1))[0],
    }

    sampled_indices = np.concatenate([
        random.sample(list(group_indices['females_mat']), 50),
        random.sample(list(group_indices['females_pat']), 50),
    ])

    subset = dn_volumes.iloc[sampled_indices].sort_index()
    X_females = np.array(subset[dn_cols])
    y_females = subset.only_mother.values

    # Fit real model
    model = LogisticRegression(solver='liblinear')
    model.fit(X_females, y_females)
    real_coefs = model.coef_[0]
    real_coefs_df = pd.DataFrame(model.coef_, columns=dn_cols)
    all_coefs_f = pd.concat([all_coefs_f, real_coefs_df], ignore_index=True)

    # Build null distribution in parallel
    null_seeds = range(0,1000)
    null_coefs = Parallel(n_jobs=6)(
        delayed(fit_null_model)(X_females, y_females, s) for s in null_seeds
    )
    null_coefs_df = pd.DataFrame(null_coefs, columns=dn_cols)

    # Compute p-values
    dict_pval = compute_pval(null_coefs_df, real_coefs)
    pval_df = pd.DataFrame({
        'coefs': list(dict_pval.keys()),
        f'pval_{seed}': list(dict_pval.values())
    })
    pval_dfs_f.append(pval_df)

# Save results
all_coefs_f.to_csv(f'{date}/brain_imaging/diagnostic_test/all_coefs_females_prevent_AD_dn_2025.csv', index=False)

# Combine p-values after loop
all_pvals_f = reduce(lambda left, right: left.merge(right, on='coefs'), pval_dfs_f)
# Save results
all_pvals_f.to_csv(f'{date}/brain_imaging/diagnostic_test/prevent_ad_dn_all_pvals_f_1000_2025.csv', index=False)

Permutations for DN female model: 100%|██████████| 1000/1000 [10:05<00:00,  1.65it/s]
